# PyGMTSAR Python Notebook S1A_Stack_CPGF_T173

#### Tested on MacOS Catalina (Python 3.9) and Debian 10 (Python 3.7)

### I'm a freelancer and that's my free time Open Source project with GPL-3.0 License. If you find it useful you are able to sponsor my projects <a href="https://www.patreon.com/bePatron?u=54500608" data-patreon-widget-type="become-patron-button">Become a Patron!</a><script async src="https://c6.patreon.com/becomePatronButton.bundle.js"></script> or order additional research, development and support on <a href="https://www.upwork.com/freelancers/~01e65e8e7221758623">Upwork</a>

### @ Alexey Pechnikov, Sep, 2021, https://github.com/mobigroup

## Check for GMTSAR installation

#### Just wait 10-30 minutes and restart the notebook if cloud installation is not ready

Use the provided Google Cloud init script for Debian 10 VM or Jupyter Notebook on Debian 10:
https://github.com/mobigroup/gmtsar/blob/master/gmtsar/sh/GMTSAR.install.debian10.sh

In [ ]:
count = !ls /usr/local | grep GMTSAR | wc -l
assert count != ['0'], \
    'Please wait until your init script complete on a cloud host or install GMTSAR manually on local host'

## Download and unpack the example, create processing directory

In [ ]:
count = !ls | grep S1A_Stack_CPGF_T173.tar.gz | wc -l
if count == ['0']:
    !wget -c http://topex.ucsd.edu/gmtsar/tar/S1A_Stack_CPGF_T173.tar.gz
    !tar xvzf S1A_Stack_CPGF_T173.tar.gz -C .
    !mkdir raw

## Define ENV Variables for Jupyter Instance

In [ ]:
import os

# use default GMTSAR installation path
GMTSAR = '/usr/local/GMTSAR'
PATH = os.environ['PATH']

if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + f':{GMTSAR}/bin/'
    %env PATH {PATH}
    %env GMTSAR {GMTSAR}

## Install Python Modules

In [ ]:
import sys
!{sys.executable} --version

In [ ]:
!{sys.executable} -m pip install --upgrade pip setuptools wheel > /dev/null
!{sys.executable} -m pip install cartopy==0.19.0.post1 > /dev/null

In [ ]:
!{sys.executable} -m pip install \
    h5py netcdf4 h5netcdf \
    rasterio rioxarray xarray numpy \
    scikit-image scipy sklearn \
    xarray dask distributed zarr \
    pandas geopandas \
    sentineleof elevation \
    matplotlib seaborn geoviews hvplot datashader bokeh \
    xmltodict joblib tqdm --upgrade 2>&1 > /dev/null

## Load and Setup Python Modules

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
# supress numpy warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# plotting modules
import hvplot.xarray  # noqa
import hvplot.pandas  # noqa
import holoviews as hv
from holoviews import opts
from bokeh.models import FixedTicker
hv.extension('bokeh', 'matplotlib')
#pd.options.plotting.backend = 'holoviews'
pd.options.plotting.backend = 'hvplot'

gstiles = hv.Tiles('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name='Google Satellite')
ottiles = hv.Tiles('https://tile.opentopomap.org/{Z}/{X}/{Y}.png', name='Open Topo')

In [ ]:
# define Pandas display settings
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# define common plot parameters
plot_opts = {'rasterize': True, 'xlabel':'Range', 'ylabel':'Azimuth', 'width':500, 'height':400}

## Load Custom Python Modules

In [ ]:
sys.path.append(os.path.join(os.environ['GMTSAR'],'gmtsar', 'py'))

from intf_ra2ll import intf_ra2ll_matrix, intf_ra2ll
from trans_ra2ll import trans_ra2ll_matrix
from PRM import PRM
from SBAS import SBAS

## Define Parameters

In [ ]:
MASTER       = '2015-04-03'
WORKDIR      = 'raw'
DATADIR      = 'raw_orig'
DEMFILE      = 'topo/dem.grd'
BASEDAYS     = 100
BASEMETERS   = 150
CORRLIMIT    = 0.10
DEFOMAX      = 0

## Init SBAS

In [ ]:
sbas = SBAS(DATADIR, DEMFILE, WORKDIR).set_master(MASTER)
sbas.to_dataframe()

In [ ]:
title = 'Sentinel1 Frame on DEM plus GCP'
#https://holoviz.org/tutorial/Composing_Plots.html
sbas.get_dem()[::4,::4].hvplot(invert=True, cmap='kbc', alpha=1, title=title) * \
    sbas.geoloc().plot.scatter(y='longitude', x='latitude', c='pixel', cmap='jet')

In [ ]:
title = 'Sentinel1 Frame DEM cropped using GCP'
#https://holoviz.org/tutorial/Composing_Plots.html
sbas.get_dem(geoloc=True)[::4,::4].hvplot(invert=True, cmap='kbc', alpha=1, title=title) * \
    sbas.geoloc().plot.scatter(y='longitude', x='latitude', c='pixel', cmap='jet')

## Stack Images (for a single subswath only)

In [ ]:
sbas.stack_parallel()

## SBAS Baseline

In [ ]:
baseline_pairs = sbas.baseline_pairs(days=BASEDAYS, meters=BASEMETERS)
baseline_pairs

## DEM in Radar Coordinates

In [ ]:
%%time

sbas.topo_ra()

In [ ]:
xr.open_dataarray(f'{WORKDIR}/topo_ra.grd').hvplot(cmap='kbc', title='Topo_ra', **plot_opts)

## Interferograms

In [ ]:
pairs = baseline_pairs[['ref_date', 'rep_date']]
pairs

In [ ]:
# we can just miss "func" argument when post-processing is not required
# define a postprocessing function for decimation, etc.
decimator = lambda dataarray: dataarray.coarsen({'y': 4, 'x': 4}, boundary='trim').median()

# default parameters: wavelength=200, psize=32, func=None (no postprocessing required)
sbas.intf_parallel(pairs, wavelength=400, func=decimator)

In [ ]:
phasefilts = sbas.open_grids(pairs, 'phasefilt')
phasefilts\
    .hvplot(by='pair', width=320, height=280, subplots=True,
            xlabel='Range', ylabel='Azimuth',
            clim=(-np.pi,np.pi), cmap='gist_rainbow_r')\
    .cols(3).opts(title='Filtered Phase, [rad]')

In [ ]:
corrs = sbas.open_grids(pairs, 'corr')
corrs\
    .hvplot(by='pair', width=320, height=280, subplots=True,
            xlabel='Range', ylabel='Azimuth',
            clim=(0, 0.8), cmap='gray')\
    .cols(3).opts(title='Correlation')

## Unwrapping

In [ ]:
# generate a custom snaphu config file and use it as argument "conf" value
# conf = self.PRM().snaphu_config(defomax=0)

# we can just miss "func" argument when post-processing is not required
# define a post-processing function to crop and interpolate low-coherence areas, etc.
#cleaner = lambda corr, unwrap: xr.where(corr>=CORRLIMIT, unwrap, np.nan)
cleaner = lambda corr, unwrap: sbas.nearest_grid(xr.where(corr>=CORRLIMIT, unwrap, np.nan))

# default parameters: threshold=0.1, conf=None, func=None (no postprocessing required)
sbas.unwrap_parallel(pairs, threshold=CORRLIMIT, func=cleaner)

In [ ]:
unwraps = sbas.open_grids(pairs, 'unwrap')
unwraps\
    .hvplot(by='pair', width=320, height=280, subplots=True, cmap='jet',
            xlabel='Range', ylabel='Azimuth',
            clim=tuple(np.nanquantile(unwraps, [0.01, 0.99])))\
    .cols(3).opts(title='Unwrapped Phase, [rad]')

## LOS Displacement

In [ ]:
los_disp_mm = sbas.los_displacement_mm(unwraps)
los_disp_mm\
    .hvplot(by='pair', width=320, height=280, subplots=True, cmap='jet',
            xlabel='Range', ylabel='Azimuth')\
    .cols(3).opts(title='LOS Displacement, [mm]')

## Bonus: Inverted Interferograms

We are able to build reverse-ordered interferograms by a simple hand move

In [ ]:
pairs_inverted = baseline_pairs[['rep_date', 'ref_date']]
pairs_inverted

In [ ]:
sbas.intf_parallel(pairs_inverted, wavelength=400, func=decimator)

In [ ]:
phasefilts_inverted = sbas.open_grids(pairs_inverted, 'phasefilt')
phasefilts_inverted\
    .hvplot(by='pair', width=320, height=280, subplots=True,
            xlabel='Range', ylabel='Azimuth',
            clim=(-np.pi,np.pi), cmap='gist_rainbow_r')\
    .cols(3).opts(title='Filtered Phase Inverted, [rad]')

## Bonus: Inverted Interferogram Unwrapping

In [ ]:
sbas.unwrap_parallel(pairs_inverted, threshold=CORRLIMIT, func=cleaner)

In [ ]:
unwraps_inverted = sbas.open_grids(pairs_inverted, 'unwrap')
unwraps_inverted\
    .hvplot(by='pair', width=320, height=280, subplots=True, cmap='jet',
            xlabel='Range', ylabel='Azimuth',
            clim=tuple(np.nanquantile(unwraps_inverted, [0.01, 0.99])))\
    .cols(3).opts(title='Unwrapped Phase Inverted, [rad]')

In [ ]:
# difference is equal to N*2*PI
np.round(np.nanmean(unwraps.values + unwraps_inverted.values)/np.pi, 3)

In [ ]:
xr.DataArray((unwraps.values + unwraps_inverted.values), coords=unwraps.coords)\
    .hvplot(by='pair', width=320, height=280, subplots=True, cmap='bwr',
            xlabel='Range', ylabel='Azimuth')\
    .cols(3).opts(title='Unwrapped Phase Direct and Inverse Difference (Unwrap Error), [rad]')

## TODO: SBAS Displacement

In [ ]:
sbas.PRM().filename[4:]

In [ ]:
#sbas.open_grids(pairs[:1], 'unwrap')[0]

def sbas_sbas(self, pairs):
    import math
    
    unwrap = self.open_grids(pairs[:1], 'unwrap')[0]
    prm = self.PRM()
    
    #N=$(wc -l intf.in   | cut -d ' ' -f1)
    #S=$(wc -l scene.tab | cut -d ' ' -f1)

    N = len(pairs)
    S = len(self.df)
    
    # TODO
    lon0 = sbas.geoloc().longitude.mean()
    lat0 = sbas.geoloc().latitude.mean()
    elevation0 = float(sbas.get_dem().sel(lat=lat0, lon=lon0, method='nearest'))
        
    #satlook=$(echo "$lon0 $lat0 $elevation" | SAT_look *.PRM)
    #look_E=$(echo "$satlook" | cut -d ' ' -f4)
    #look_N=$(echo "$satlook" | cut -d ' ' -f5)
    #look_U=$(echo "$satlook" | cut -d ' ' -f6)
    
    satlook = !cd raw && echo {lon0} {lat0} {elevation0} | SAT_look {self.PRM().filename[4:]}
    satlook = str(satlook)[2:-2].split(' ')
    print ('satlook', satlook)
    _,_,_,look_E,look_N,look_U = satlook
    print ('look_E,look_N,look_U', look_E,look_N,look_U )
    incidence = math.atan2(math.sqrt(float(look_E)**2 + float(look_N)**2), float(look_U))*180/np.pi

    xdim, ydim = unwrap.shape
    xmin = int(unwrap.x.min())
    xmax = int(unwrap.x.max())
    near_range, rng_samp_rate, wavelength = prm.get('near_range', 'rng_samp_rate', 'radar_wavelength')
    
    # calculation below requires bc utility
    rng_pixel_size = 300000000 / rng_samp_rate / 2
    rng = np.round(rng_pixel_size * (xmin+xmax) /2 + near_range)
    
    return N, S, xdim, ydim, rng, incidence, wavelength

SBAS.sbas = sbas_sbas

# for scene.tab
# sbas.baseline_table()

#sbas intf.tab scene.tab 9 5 $xdim $ydim -smooth 1.0 -wavelength 0.0554658 -incidence 30 -range 800184.946186 -rms -dem

sbas.sbas(pairs)